In [10]:
pip install requests

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\LG\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


In [31]:
pip install tweepy

^C
Note: you may need to restart the kernel to use updated packages.


In [32]:
import requests
import json
import pandas as pd
import numpy as np
import datetime
from datetime import datetime
import time
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry
import tweepy

### 리퀘스트 모듈을 이용한 API 호출

In [2]:
# get메소드 : 생성된 순서대로 저장소 목록을 제공함
# api 호출 성공
response = requests.get('https://api.github.com/repositories', 
headers={'Accept': 'application.vnd.github.v3+json'})
print(response.status_code)

200


In [3]:
print(response.encoding)
print(response.headers['Content-Type'])
print(response.headers['server'])

utf-8
application/json; charset=utf-8
GitHub.com


In [4]:
response.headers

{'Server': 'GitHub.com', 'Date': 'Fri, 30 Jun 2023 00:27:14 GMT', 'Content-Type': 'application/json; charset=utf-8', 'Cache-Control': 'public, max-age=60, s-maxage=60', 'Vary': 'Accept, Accept-Encoding, Accept, X-Requested-With', 'ETag': 'W/"b3f394bd2befbc0f6bd0a4c1da7e01cf55798d3fd165f3571c16135b5481cd08"', 'X-GitHub-Media-Type': 'github.v3; format=json', 'Link': '<https://api.github.com/repositories?since=369>; rel="next", <https://api.github.com/repositories{?since}>; rel="first"', 'x-github-api-version-selected': '2022-11-28', 'Access-Control-Expose-Headers': 'ETag, Link, Location, Retry-After, X-GitHub-OTP, X-RateLimit-Limit, X-RateLimit-Remaining, X-RateLimit-Used, X-RateLimit-Resource, X-RateLimit-Reset, X-OAuth-Scopes, X-Accepted-OAuth-Scopes, X-Poll-Interval, X-GitHub-Media-Type, X-GitHub-SSO, X-GitHub-Request-Id, Deprecation, Sunset', 'Access-Control-Allow-Origin': '*', 'Strict-Transport-Security': 'max-age=31536000; includeSubdomains; preload', 'X-Frame-Options': 'deny', 'X-

In [5]:
# 응답이 json객체이므로 json()으로 응답 읽기
# 각 요소에 저장소가 저장된 리스트 객체를 생성
print(json.dumps(response.json()[0], indent=2)[:200])

{
  "id": 1,
  "node_id": "MDEwOlJlcG9zaXRvcnkx",
  "name": "grit",
  "full_name": "mojombo/grit",
  "private": false,
  "owner": {
    "login": "mojombo",
    "id": 1,
    "node_id": "MDQ6VXNlcjE=",



In [6]:
# 검색 api
# 422 : 요청은 정확하지만 서버가 요청을 처리할 수 없음
# 검색어 매개변수를 제공하지 않았기 때문
response = requests.get('https://api.github.com/search/repositories')
print (response.status_code)

422


In [7]:
# 파이썬으로 작성된 데이터 과학과 관련된 깃허브 저장소 찾아보기
# text-match+json : 원하는 메타데이터와 json 형식의 결과를 얻기 위해
response = requests.get('https://api.github.com/search/repositories',
                        params={'q': 'data_science+language:python'},
                        headers={'Accept': 'application/vnd.github.v3.text-match+json'})

print(response.status_code)

200


In [8]:
# 검색에서 반환된 상위 5개 저장소의 이름 나열
from IPython.display import Markdown, display  ###
def printmd(string):  ###
    display(Markdown(string))  ###

for item in response.json()['items'][:5]:
    printmd('**' + item['name'] + '**' + ': repository ' +
            item['text_matches'][0]['property'] + ' - \"*' +
            item['text_matches'][0]['fragment'] + '*\" matched with ' + '**' +
            item['text_matches'][0]['matches'][0]['text'] + '**')

**data-science-from-scratch**: repository description - "*code for Data Science From Scratch book*" matched with **Data Science**

**data-science-blogs**: repository description - "*A curated list of data science blogs*" matched with **data science**

**galaxy**: repository description - "*Data intensive science for everyone.*" matched with **Data**

**DataCamp**: repository description - "*DataCamp data-science courses*" matched with **data**

**data-scientist-roadmap**: repository description - "*Toturials coming with the "data science roadmap" picture.*" matched with **data science**

In [9]:
# 저장소의 댓글을 모니터링하고 커뮤니티 지침을 준수하는지 확인
response = requests.get(
    'https://api.github.com/repos/pytorch/pytorch/issues/comments')

print('Response Code', response.status_code)
print('Number of comments', len(response.json()))

Response Code 200
Number of comments 30


#### 페이지 매기기

In [10]:
# next : 다음 결과 30개를 포함하는 다음 페이지의 url
# last : 총 검색 결과가 몇 개 있는지를 나타내는 마지막 페이지의 링크
response.links

{'next': {'url': 'https://api.github.com/repositories/65600975/issues/comments?page=2',
  'rel': 'next'},
 'last': {'url': 'https://api.github.com/repositories/65600975/issues/comments?page=1000',
  'rel': 'last'}}

In [11]:
# 2020년 7월 이후의 댓글만 가져오기
# 날짜는 ISO 8601형식으로 지정하고 since 키워드를 사용해 매개변수로 제공
def get_all_pages(url, params=None, headers=None):
    output_json = []
    response = requests.get(url, params=params, headers=headers)
    if response.status_code == 200:
        output_json = response.json()
        if 'next' in response.links:
            next_url = response.links['next']['url']
            if next_url is not None:
                output_json += get_all_pages(next_url, params, headers)
    return output_json


out = get_all_pages(
    "https://api.github.com/repos/pytorch/pytorch/issues/comments",
    params={
        'since': '2020-07-01T10:00:01Z',
        'sorted': 'created',
        'direction': 'desc'
    },
    headers={'Accept': 'application/vnd.github.v3+json'})
df = pd.DataFrame(out)

In [12]:
# pd.set_option('display.max_colwidth', -1) ###
print (df['body'].count())
df[['id','created_at','body']].sample(1, random_state=42)

990


,id,created_at,body
213,1613470964,2023-06-29T16:08:25Z,<!-- drci-comment-start -->\n\n## :link: Helpf...


#### 속도 제한

In [13]:
# 헤더만 검색하는 api 호출
# X-Ratelimit-Limit : 단위 시간(1시간) 당 요청 수
# X-Ratelimit-Remaining : 현재 속도 제한을 넘기지 않고 보낼 수 있는 요청 수
# X-RateLimit-Reset : 요청 수가 재설정되는 시간
response = requests.head(
    'https://api.github.com/repos/pytorch/pytorch/issues/comments')

print('X-Ratelimit-Limit', response.headers['X-Ratelimit-Limit'])
print('X-Ratelimit-Remaining', response.headers['X-Ratelimit-Remaining'])

# UTC 시간을 사람이 읽을 수 있는 형식으로 변환함.
print(
    'Rate Limits reset at',
    datetime.datetime.fromtimestamp(int(
        response.headers['X-RateLimit-Reset']
    )).strftime('%c')
)

X-Ratelimit-Limit 60
X-Ratelimit-Remaining 23
Rate Limits reset at Fri Jun 30 10:27:14 2023


In [17]:
# 요청이 제한 시간을 초과하지 않도록 
# 속도를 늦춰 요청을 일정 시간 간격으로 보내도록 조정함.
# sleep 기능 : 남은 요청을 남은 시간에 균등하게 분배

def handle_rate_limits(response) : 
    now = datetime.now()
    reset_time = datetime.fromtimestamp(
        int(response.headers['X-RateLimit-Reset']))
    remaining_requests = response.headers['X-Ratelimit-Remaining']
    remaining_time = (reset_time - now).total_seconds()
    intervals = remaining_time / (1.0 + int(remaining_requests))
    print('Sleeping for', intervals)
    time.sleep(intervals)
    return True

In [19]:
#  재시도 기능을 구현하는 함수
# retry_strategy : 재시도 기능(여기서는 재시도 5번 시행)
# HTTPAdapter : 기본 HTTP연결을 세밀하게 제어
# 재시도의 오류 코드 : 500, 503, 504
# backoff_factor : 두 번째 시도 이후 각 시도 사이의 시간 지연을 기하급수적으로 증가시킴.
retry_strategy = Retry(
    total = 5,
    status_forcelist = [500,503,504],
    backoff_factor = 1
)

retry_adapter = HTTPAdapter(max_retries=retry_strategy)

http = requests.Session()
http.mount('https://', retry_adapter)
http.mount('http://', retry_adapter)

response = http.get('https://api.github.com/search/repositories',
                    params={'q':'data_science+language:python'})

for item in response.json()['items'][:5] : 
    print(item['name'])

data-science-from-scratch
data-science-blogs
galaxy
DataCamp
data-scientist-roadmap


In [25]:
# 페이지 매기기, 속도 제한, 재시도 한꺼번에 하도록 수정
retry_strategy = Retry(
    total = 5,
    status_forcelist = [500,503,504],
    backoff_factor = 1
)

retry_adapter = HTTPAdapter(max_retries=retry_strategy)

http = requests.Session()
http.mount('https://', retry_adapter)
http.mount('http://', retry_adapter)

def get_all_pages(url, param=None, header=None) : 
    output_json = []
    response = http.get(url, params=param, headers=header)
    if response.status_code == 200 : 
        output_json = response.json()
        if 'next' in response.links : 
            next_url = response.links['next']['url']
            if (next_url is not None) and (handle_rate_limits(response)) : 
                output_json += get_all_pages(next_url, param, header)

    return output_json

In [26]:
out = get_all_pages('https://api.github.com/repos/pytorch/pytorch/issues/comments',
                    param={'since': '2020-04-01T00:00:01Z'})
df = pd.DataFrame(out)

### 트위피를 사용한 트위터 데이터 추출

In [42]:
app_api_key = 'VJzNiGcU2uzgMWiQqOzrIGwFW'
app_api_secret_key = 'jEQbPBkhg5MeFmmRgBw2LDkw6plazdMD5u6YL25gu2HfFkApXG'

auth = tweepy.AppAuthHandler(app_api_key, app_api_secret_key)
api = tweepy.API(auth)

print ('API Host', api.host)
# print ('API Version', api.api_root)

API Host api.twitter.com


#### 검색 API에서 데이터 추출

In [45]:
# Cursor 객체로 페이지 매기기를 반복할 수 있음
pd.set_option('display.max_colwidth', None)

search_term = 'cryptocurrency'

tweets = tweepy.Cursor(api.search_tweets, 
                       q = search_term,
                       lang='en').items(100)

retrieved_tweets = [tweets._json for tweet in tweets]
df = pd.json_normalize(retrieved_tweets)

df[['text']].sample(3)

Forbidden: 403 Forbidden
453 - You currently have access to a subset of Twitter API v2 endpoints and limited v1.1 endpoints (e.g. media post, oauth) only. If you need access to this endpoint, you may need a different access level. You can learn more here: https://developer.twitter.com/en/portal/product